# Modèles Factoriels : CAPM & Fama-French

Permet d’estimer les sensibilités des actifs à différents facteurs de marché à l’aide de modèles linéaires :

- CAPM : sensibilité au marché (beta)
- Fama-French 3 facteurs : marché, taille (SMB), valeur (HML)

Les résultats incluent les coefficients, les p-values, le R² et l’alpha.

# CAPM

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sys

sys.path.append("../src")

from factor_models import run_capm, expected_returns_capm
from data_loader import get_data_adj, get_benchmark

tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "DIS"]
prices = get_data_adj(tickers, start="2024-08-01", end="2025-07-31")

#Ici nous prenons le S&P500 comme benchmark
import yfinance as yf
benchmark = get_benchmark(start="2024-08-01", end="2025-07-31")


# Calcul des betas CAPM
betas = run_capm(prices, benchmark, risk_free_rate=0.02)

# Calcul des rendements attendus
exp_returns = expected_returns_capm(betas, expected_market_return=0.08, risk_free_rate=0.02)

# Affichage
print("Betas CAPM :")
print(betas.round(4))
print("\nRendements attendus via CAPM :")
print(exp_returns.round(4))


[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed

Betas CAPM :
AAPL     1.2256
DIS      1.0179
GOOGL    1.0318
MSFT     0.9597
TSLA     2.3786
dtype: float64

Rendements attendus via CAPM :
AAPL     0.0935
DIS      0.0811
GOOGL    0.0819
MSFT     0.0776
TSLA     0.1627
dtype: float64


## Résultats du modèle CAPM

Le modèle CAPM permet d’estimer la sensibilité de chaque actif au marché via le **beta**, et d’en déduire le **rendement attendu** en fonction du risque systématique.

### Betas CAPM :
- Un **beta > 1** indique une volatilité supérieure à celle du marché (ex: TSLA).
- Un **beta < 1** indique une volatilité inférieure (ex: MSFT).
- Un **beta ≈ 1** signifie que l’actif évolue globalement comme le marché (ex: DIS).

### Rendements attendus :

Ces rendements permettent d’évaluer si un actif est **surperformant ou sous-performant** par rapport à ce que le modèle prédit.

# FAMA-FRENCH

In [1]:
import pandas as pd
import numpy as np
import sys

sys.path.append("..")
from src.data_loader import get_data_adj, get_ff_factors
from src.factor_models import run_fama_french_stats

# Définir les tickers et la période
tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "DIS"]
start_date = "2023-08-01"
end_date = "2024-07-31"

start_date_ff = "20230801"
end_date_ff = "20240731"
# Télécharger les prix ajustés des actifs
prices = get_data_adj(tickers, start=start_date, end=end_date)

# Calcul des rendements journaliers des actifs
returns = np.log(prices / prices.shift(1)).dropna()

# Choisir la fréquence des facteurs (monthly ou annual)
frequency = 'monthly'

# Récupérer les facteurs Fama-French avec la bonne fréquence
factors_daily = get_ff_factors(start_date_ff, end_date_ff, frequency=frequency)

# Estimation Fama-French avec les facteurs réels
results_ff = run_fama_french_stats(returns, factors_daily, risk_free_rate=0.02)

# Affichage des résultats
print("\nRésultats Fama-French 3 Facteurs (facteurs réels Fama-French) :")
print(results_ff.round(4))


[*********************100%***********************]  5 of 5 completed


Résultats Fama-French 3 Facteurs (facteurs réels Fama-French) :
        Alpha  Beta_MKT  Beta_SMB  Beta_HML  R_squared   p_MKT   p_SMB   p_HML
AAPL   0.0047    0.0311    0.1856    0.0073     0.3461  0.8063  0.3707  0.9733
DIS   -0.0069    0.1321   -0.0129   -0.0879     0.4439  0.2035  0.9285  0.5905
GOOGL  0.0099   -0.0307    0.0412   -0.2195     0.2372  0.8206  0.8444  0.3735
MSFT   0.0102    0.0609    0.0000   -0.1060     0.1546  0.6291  1.0000  0.6269
TSLA  -0.0072    0.2074    0.3096    0.1501     0.3979  0.4930  0.5081  0.7682
